# Capstone Project Notebook
### (To get IBM Data Science Professional Certificate)

![alt text](Img01.PNG "Data Science")

This notebook will show the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

> Given a city like the City of Toronto, you will segment it into different neighborhoods using the geographical coordinates of the center of each neighborhood, and then using a combination of location data and machine learning, you will group the neighbourhoods into clusters 



### First section

In [1]:
import pandas as pd
import numpy as np

# Read source
tmp = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# The first webpage table has the needed information for this project
postal_codes_tmp = tmp[0]

# Filter records which Borough == Not assigned
na_filter = postal_codes_tmp['Borough'] != 'Not assigned'
filtered_postal_codes = postal_codes_tmp[na_filter]

# More than one neighborhood can exist in one postal code area. For such cases, neighborhoods will be shown in the same row separated with a comma (e.g. M1C, M3J, M5A, etc.)
postal_codes = filtered_postal_codes.groupby(['Postcode', 'Borough']).agg([(', '.join)]).reset_index()

# Rename column headers
postal_codes.columns = ['PostalCode', 'Borough', 'Neighborhood']

# Whenever the Neighborhood is Not assigned, the Borough name should be used as the Neighborhood value (e.g. M7A)
postal_codes.Neighborhood.replace('Not assigned',postal_codes.Borough,inplace=True)

print('Pandas is awesome to read html tables!')
postal_codes.head()

Pandas is awesome to read html tables!


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [2]:
# Regardless the information is shown above, here you can check the number of the dataframe rows
print(str(postal_codes.shape[0]) + ' dataframe rows.')

103 dataframe rows.


### Second section

In [5]:
# Read coordinates source
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

# Rename column headers
coordinates.columns = ['PostalCode', 'Latitude', 'Longitude']

toronto_data = pd.merge(postal_codes, coordinates, how='inner', left_on = 'PostalCode', right_on = 'PostalCode')
print(toronto_data.shape)
toronto_data.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Third section

#### Installing all the needed libraries

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Verifying number of boroughs and neighborhoods

In [9]:
print('The Toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The Toronto dataframe has 11 boroughs and 103 neighborhoods.


#### Get Toronto latitude and longitud to build the map

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Credentials setup and function to get venues by neighborhood in Toronto

In [15]:
# Define Foursquare Credentials and Version
CLIENT_ID = '020DHIJQ5OJ4YZ12HXXY4O0D33CXV4OT0QXK25QO3Y03IK1I'
CLIENT_SECRET = 'P3SIW32METMPEVCC1WEZ3DXWQEFVGA2YZC5ELTDWT2FSYVW4'
VERSION = '20180605'
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Get Toronto venues using defined function
##### 278 uniques categories in 2,264 venues (7 columns)

In [24]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

print(str(toronto_venues.shape[0]) + ' venues with ' + str(toronto_venues.shape[1]) + ' columns')
#toronto_venues.head()

#toronto_venues.groupby('Neighborhood').count()

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

##### One hot encoding
###### 278 uniques categories in 2,264 venues (7 columns). 100 different neighborhoods

In [25]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# toronto_onehot.head()

print(toronto_onehot.shape)

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
# print(toronto_grouped)

print(toronto_grouped.shape)

(2264, 278)
(100, 278)


##### 5 most common venues per neighborhood

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Agincourt----
                venue  freq
0        Skating Rink  0.25
1      Breakfast Spot  0.25
2              Lounge  0.25
3      Clothing Store  0.25
4  Miscellaneous Shop  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground  0.33
1                        Gym  0.33
2                       Park  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.22
1              Pharmacy  0.11
2  Fast Food Restaurant  0.11
3        Sandwich Place  0.11
4   Fried Chicken Joint  0.11


----Alderwood, Long Branch----
         venue  fre

##### Function to sort venues in descending order

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Top 10 venues per neighborhood

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Sushi Restaurant,Asian Restaurant,Bakery,Hotel
1,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Gym,Park,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Pizza Place,Pharmacy,Dumpling Restaurant,Drugstore
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Sandwich Place,Pool,Pub,Discount Store,Department Store


##### Run k-means to cluster the neighborhood into 5 clusters

In [53]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

##### Top 10 venues per neighborhood including cluster 

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterL', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.fillna(0, inplace=True) # For some reason the labels were converted to float in the previous step
# toronto_merged.dtypes

toronto_merged['ClusterL'] = toronto_merged['ClusterL'].astype(int)

##### Create map to show clusters

In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterL']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Examine clusters
###### C1 : City zone

In [67]:
toronto_merged.loc[toronto_merged['ClusterL'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
2,Scarborough,0,Electronics Store,Spa,Rental Car Location,Mexican Restaurant,Breakfast Spot,Medical Center,Pizza Place,Intersection,Drugstore,Donut Shop
3,Scarborough,0,Coffee Shop,Korean Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
4,Scarborough,0,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Dog Run,Diner,Discount Store
5,Scarborough,0,Women's Store,Health & Beauty Service,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
6,Scarborough,0,Coffee Shop,Playground,Discount Store,Department Store,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Dive Bar,Dog Run
7,Scarborough,0,Bakery,Bus Line,Park,Metro Station,Bus Station,Soccer Field,Fast Food Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
8,Scarborough,0,Motel,American Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Women's Store
9,Scarborough,0,General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
10,Scarborough,0,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Gaming Cafe,Latin American Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


###### C2 : Cafeteria zone

In [71]:
toronto_merged.loc[toronto_merged['ClusterL'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,1,Cafeteria,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Women's Store,Department Store


###### C3 : Gym and park zone

In [70]:
toronto_merged.loc[toronto_merged['ClusterL'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2,Gym,Park,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
23,North York,2,Park,Convenience Store,Bank,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Women's Store
30,North York,2,Park,Airport,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
40,East York,2,Park,Convenience Store,Pizza Place,Coffee Shop,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
50,Downtown Toronto,2,Park,Playground,Building,Trail,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
74,York,2,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Golf Course,Greek Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
79,North York,2,Park,Bakery,Construction & Landscaping,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
90,Etobicoke,2,River,Park,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
91,Etobicoke,2,Park,Pool,Baseball Field,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
98,York,2,Park,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


###### C4 : Food zone

In [69]:
toronto_merged.loc[toronto_merged['ClusterL'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,North York,3,Empanada Restaurant,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


###### C5 : Women's zone due stores and dessert/donut shops

In [68]:
toronto_merged.loc[toronto_merged['ClusterL'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4,Bar,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
